<a href="https://colab.research.google.com/github/HUMINTING/callte-detetion-detail/blob/main/pytorch_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pretrainedmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=af131c42d157c806e12843bafba315dba021f44c5f372afa65284e2ee5a707b4
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [ ]:
# coding: utf-8
%matplotlib inline
import os
from tqdm import tqdm
import argparse
import time
import datetime as dt
import json
from PIL import Image

import torch
import torch.nn as nn
from torchvision import models
import pretrainedmodels
import torch.nn.functional as F

import pickle
import math
import glob
from sklearn import random_projection
from sklearn.manifold import TSNE
import seaborn as sns
import torch.nn as nn

import numpy as np
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

import torch
from torchvision import datasets, transforms

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt




In [ ]:
class Net(nn.Module):
    def __init__(self, name):
        super(Net, self).__init__()
        self.name = name
        if name == "vgg16":
            self.model = models.vgg16(pretrained=True)
            layers = list(self.model.classifier.children())[:-2]
            self.model.classifier = nn.Sequential(*layers)
            self.fc1 = nn.Linear(4096, 1024)
            self.drop1 = nn.Dropout(0.5)
            self.fc2 = nn.Linear(1024, 2)
            
        elif name == "resnet":
            self.model = models.resnet50(pretrained=True)
            fc_layer = nn.Sequential(
                nn.Linear(2048, 512),
                nn.ReLU(True),
                nn.Dropout(0.5),
                nn.Linear(512, 2)
            )
            self.model.fc = fc_layer
            
        elif name == "densenet":
            self.model = models.densenet161(pretrained=True)
            fc_layer = nn.Sequential(
                nn.Linear(2208, 512),
                nn.ReLU(True),
                nn.Dropout(0.5),
                nn.Linear(512, 2)
            )
            self.model.classifier = fc_layer

    def forward(self, x):
        if self.name == "vgg16":
            h = self.model(x)
            h = F.relu(self.drop1(self.fc1(h)))
            h = self.fc2(h)
        elif self.name == "resnet":
            h = self.model(x)
        elif self.name == "densenet":
            h = self.model(x)
        return h

In [ ]:
import os
home = os.path.expanduser('~')

##数据集的类别
NUM_CLASSES = 206

#训练时batch的大小
BATCH_SIZE = 32

#网络默认输入图像的大小
INPUT_SIZE = 300
#训练最多的epoch
MAX_EPOCH = 100
# 使用gpu的数目
GPUS = 2
# 从第几个epoch开始resume训练，如果为0，从头开始
RESUME_EPOCH = 0

WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
# 初始学习率
LR = 1e-3

BASE =  '/content/drive/MyDrive/estrus_20191230/1/'

# 训练好模型的保存位置
SAVE_FOLDER = '/content/drive/MyDrive/estrus_20191230/save_weight/'
#数据集的存放位置
TRAIN_LABEL_DIR =BASE + 'train.txt'     
VAL_LABEL_DIR = BASE + 'val.txt'
TEST_LABEL_DIR = BASE + 'test.txt'
#训练完成，权重文件的保存路径,默认保存在trained_model下
TRAINED_MODEL = BASE + 'weights/resnet/epoch_40.pth'

# 采用的模型名称
model_name = 'resnext'

In [ ]:
#transform
import random
import math
import torch

from PIL import Image, ImageOps, ImageFilter
from torchvision import transforms

class Resize(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = size
        self.interpolation = interpolation

    def __call__(self, img):
        # padding
        ratio = self.size[0] / self.size[1]
        w, h = img.size
        if w / h < ratio:
            t = int(h * ratio)
            w_padding = (t - w) // 2
            img = img.crop((-w_padding, 0, w+w_padding, h))
        else:
            t = int(w / ratio)
            h_padding = (t - h) // 2
            img = img.crop((0, -h_padding, w, h+h_padding))

        img = img.resize(self.size, self.interpolation)

        return img

class RandomRotate(object):
    def __init__(self, degree, p=0.5):
        self.degree = degree
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            rotate_degree = random.uniform(-1*self.degree, self.degree)
            img = img.rotate(rotate_degree, Image.BILINEAR)
        return img

class RandomGaussianBlur(object):
    def __init__(self, p=0.5):
        self.p = p
    def __call__(self, img):
        if random.random() < self.p:
            img = img.filter(ImageFilter.GaussianBlur(
                radius=random.random()))
        return img



mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
def get_train_transform(mean=mean, std=std, size=0):
    train_transform = transforms.Compose([
        Resize((int(size * (256 / 224)), int(size * (256 / 224)))),
        transforms.RandomCrop(size),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        RandomRotate(15, 0.3),
        # RandomGaussianBlur(),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])
    return train_transform

def get_test_transform(mean=mean, std=std, size=0):
    return transforms.Compose([
        Resize((int(size * (256 / 224)), int(size * (256 / 224)))),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])

def tta_test_transform(mean=mean, std=std, size=0):
    return transforms.Compose([
        Resize((int(size * (256 / 224)), int(size * (256 / 224)))),
        # transforms.CenterCrop(size),
        transforms.RandomCrop(size),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        RandomRotate(15, 0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from PIL import Image


import sys 
sys.path.append("..") 




input_size = INPUT_SIZE
batch_size = BATCH_SIZE

# 构建数据提取器，利用dataloader
# 利用torchvision中的transforms进行图像预处理
#cfg为config文件，保存几个方便修改的参数

class  SelfCustomDataset(Dataset):
    def __init__(self, label_file, imageset):
        '''
        img_dir: 图片路径：img_dir + img_name.jpg构成图片的完整路径      
        '''
        # 所有图片的绝对路径
        with open(label_file, 'r') as f:
            #label_file的格式， （label_file image_label)
            self.imgs = list(map(lambda line: line.strip().split(' '), f))
      # 相关预处理的初始化
      #   self.transforms=transform
        self.img_aug=True
        if imageset == 'train':
            self.transform= get_train_transform(size=INPUT_SIZE)
        else:
            self.transform = get_test_transform(size = INPUT_SIZE)
        self.input_size = INPUT_SIZE

    def __getitem__(self, index):
        img_path, label = self.imgs[index]
        # print(img_path)
        img = Image.open(img_path).convert('RGB')
        if self.img_aug:
            img =self.transform(img)


        else:
            img = np.array(img)
            img = torch.from_numpy(img)

        return img, torch.from_numpy(np.array(int(label)))
 
    def __len__(self):
        return len(self.imgs)



##ImageFolder对象可以将一个文件夹下的文件构造成一类
#所以数据集的存储格式为一个类的图片放置到一个文件夹下
#然后利用dataloader构建提取器，每次返回一个batch的数据，在很多情况下，利用num_worker参数
#设置多线程，来相对提升数据提取的速度

train_label_dir = TRAIN_LABEL_DIR
train_datasets = SelfCustomDataset(train_label_dir, imageset='train')
train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True, num_workers=2)

val_label_dir = VAL_LABEL_DIR
val_datasets = SelfCustomDataset(val_label_dir, imageset='test')
val_dataloader = torch.utils.data.DataLoader(val_datasets, batch_size=batch_size, shuffle=True, num_workers=2)


##进行数据提取函数的测试
if __name__ =="__main__":

    for images, labels in train_dataloader:
        print(labels)


FileNotFoundError: ignored

In [ ]:
# -*-coding:utf-8-*-


import os
import os.path


def write_txt(content, filename, mode='w'):
    """保存txt数据
    :param content:需要保存的数据,type->list
    :param filename:文件名
    """
    with open(filename, mode) as f:
        for line in content:
            str_line = ""
            for col, data in enumerate(line):
                if not col == len(line) - 1:
                    # 以空格作为分隔符
                    str_line = str_line + str(data) + " "
                else:
                    # 每行最后一个数据用换行符“\n”
                    str_line = str_line + str(data) + "\n"
            f.write(str_line)


def get_files_list(dir):
    '''
    实现遍历dir目录下,所有文件(包含子文件夹的文件)
    :param dir:指定文件夹目录
    :return:包含所有文件的列表->list
    '''
    # parent:父目录, filenames:该目录下所有文件夹,filenames:该目录下的文件名
    files_list = [] #写入文件的数据
    for parent, dirnames, filenames in os.walk(dir):
        for filename in filenames:
            print("parent is: " + parent)
            print("filename is: " + filename)
            print(os.path.join(parent, filename).replace('\\','/'))  # 输出rootdir路径下所有文件（包含子文件）信息
            curr_file = parent.split(os.sep)[-1]	#获取正在遍历的文件夹名（也就是类名）
			#根据class名确定labels
            if curr_file == "mt":
                labels = 0
            elif curr_file == "one":
                labels = 1
            elif curr_file == "multiple":
                labels = 2
            elif curr_file == "garbage":
                labels = 3

       		  dir_path = parent.replace('\\','/').split('/')[-2]  #train?val?test?

            curr_file = os.path.join(dir_path, curr_file)  #相对路径

            files_list.append([os.path.join(curr_file, filename).replace('\\','/'), labels])	#相对路径+label


    		#写入csv文件
            path = "%s" % os.path.join(curr_file, filename).replace('\\','/')
            label = "%d" % labels
            list = [path, label]
            data = pd.DataFrame([list])
            if dir == '/content/drive/MyDrive/estrus_20191230/1/train':
                data.to_csv("./content/drive/MyDrive/estrus_20191230/1/train.csv", mode='a', header=False, index=False)
            elif dir == '/content/drive/MyDrive/estrus_20191230/1/valid':
                data.to_csv("/content/drive/MyDrive/estrus_20191230/1/val.csv", mode='a', header=False, index=False)
            elif dir == '/content/drive/MyDrive/estrus_20191230/1/test':
                data.to_csv("/content/drive/MyDrive/estrus_20191230/1/test.csv", mode='a', header=False, index=False)
    
    return files_list


if __name__ == '__main__':

    import pandas as pd
    #先生成两个csv文件夹
    df = pd.DataFrame(columns=['path', 'label'])
    df.to_csv("/content/drive/MyDrive/estrus_20191230/1/train.csv", index=False)

    df2 = pd.DataFrame(columns=['path', 'label'])
    df2.to_csv("/content/drive/MyDrive/estrus_20191230/1/val.csv", index=False)

    df3 = pd.DataFrame(columns=['path', 'label'])
    df3.to_csv("/content/drive/MyDrive/estrus_20191230/1/test.csv", index=False)

    #写入txt文件
    train_dir = '/content/drive/MyDrive/estrus_20191230/1/train'
    train_txt = '/content/drive/MyDrive/estrus_20191230/1/train.txt'
    train_data = get_files_list(train_dir)
    write_txt(train_data, train_txt, mode='w')

    val_dir = '/content/drive/MyDrive/estrus_20191230/1/valid'
    val_txt = '/content/drive/MyDrive/estrus_20191230/1/val.txt'
    val_data = get_files_list(val_dir)
    write_txt(val_data, val_txt, mode='w')

    train_dir = '/content/drive/MyDrive/estrus_20191230/1/test'
    train_txt = '/content/drive/MyDrive/estrus_20191230/1/test.txt'
    train_data = get_files_list(train_dir)
    write_txt(train_data, train_txt, mode='w')




TabError: ignored